In [4]:
import pandas as pd
import numpy as np

from datetime import datetime
import re

In [5]:
# импорт моделей
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.base import clone

#графики
import pylab as pl
import matplotlib.pyplot as plt

In [12]:
def del_duplicate_col(df):
    '''удаление повторяющихся строк по столбцу client_id'''
    print(f'до: {df.shape[0]}')
    df.reset_index(inplace=True, drop=False)
    duplicate_index = df[df['client_id'].duplicated()].index
    duplicate_index.append(duplicate_index-1)
    df = df.drop(duplicate_index)
    df.set_index('client_id', inplace=True)
    print(f'после: {df.shape[0]}')
    return df


def f_create_agegroup(df_features):
    '''создание фич возрастных категорий'''
    df_features['age'] = df_features['age'].apply(lambda x: 99 if x<0 or x>90 else x)
    df_features['age_group_kk'] = pd.cut(df_features['age'], [0,10,20,30,40,50,60,70,80,90]) 
    df_age_group = pd.get_dummies(df_features['age_group_kk'],
                                  prefix="age_group_kk",
                                  drop_first=True)
    print('Создание категорий возрастов завершено')
    return df_age_group


def clear_bad_col(df):
    '''Очистка названий колонок от не читаемых символов'''
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    df.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col 
                  for col in df.columns.values]
    return df


def f_normaliz_df(df, col):
    '''нормализация количественных данных'''
    df[col] = (df[col] - df[col].mean()) / df[col].std()
    print('Данные нормализованы')
    return df


def f_is_alhocol(df):
    #приобретался ли алкоголь
    df['is_alhocol'] = df['alhocol'].apply(lambda x: 0 if x==0 else 1)
    return df


def f_group_future(df):
    df['sum_quantity'] = df['sum'] / df['quantity']
    return df

In [ ]:
def f_processing_purchase_sum_col(df_purchases):
    '''извличение из таблицы заказы
    количество всех покупок, максимальная сумма покупки, минимальная, средняя''' 
    df_ps = df_purchases.groupby('client_id')['purchase_sum'].agg(
        [('purchase_all_sum', lambda a: sum(list(a.unique()))), 
         ('purchase_all_min', 'min'),
         ('purchase_all_max', 'max'),
         ('purchase_all_mean', lambda a: a.unique().mean()),
         ('purchase_all_std', lambda a: a.unique().std())])
    print('Извличение признаков из purchase_sum завершено')
    return df_ps


def f_processing_rpr_column(df_purchases):
    '''regular_points_received + store'''
    df_rpr = df_purchases.groupby('client_id')['regular_points_received'].agg(
        [('rpr_median', lambda a: a.median()), 
         ('rpr_min', 'min'),
         ('rpr_max', 'max'),
         ('rpr_mean', lambda a: a.unique().mean()),
         ('rpr_std', lambda a: a.unique().std())])
    print('Обработка regular_points_received завершена')
    return df_rpr


def f_onestore(df_purchases):
    '''Подсчет количества посещеных магазинов'''
    df_store = df_purchases.groupby('client_id')['store_id'].agg([('store_count', lambda a: len(set(a)))])
    df_store['is_one_store'] = df_store['store_count'].apply(lambda x: 1 if x == 1 else 0)
    print('Обработка store завершена')
    return df_store

In [13]:
def predict_broke_age(features_set):
    '''Это функция тренирует модель на предсказание возраста и делает предсказание ошибочного возраста.
    Под ошибочным понимается тот, который выбивается из интервала 14-95 лет'''

    print('Запущена модель для корректировки неправильного возраста клиентов')
    broke_age_index = features_set[(features_set['age']<14) | (features_set['age']>95)].index

    X = features_set[~features_set.index.isin(broke_age_index)].drop(['age', 'client_id'], axis=1)
    y = features_set[~features_set.index.isin(broke_age_index)]['age']
    params = {'n_jobs': -1,
              'seed': 42,
              'boosting_type': 'gbdt',
              'objective': 'regression',
              'metric': 'rmse',
              'verbose': -1}
    
    train_set = lgb.Dataset(X.iloc[:30000], y[:30000])
    val_set = lgb.Dataset(X.iloc[-5000:], y[-5000:])
    model = lgb.train(params=params,
                      train_set = train_set,
                      valid_sets = [train_set, val_set],
                      num_boost_round=5000,
                      early_stopping_rounds=100,
                      verbose_eval=False)
    
    age_predict = model.predict(features_set.loc[broke_age_index, :].drop(['age', 'client_id'], axis=1),\
                                num_iteration = model.best_iteration)
    
    features_set.loc[broke_age_index, 'age'] = age_predict.astype(int)
    print('Корректрировка возростов выполнена')
    return features_set

# Чтение данных

In [ ]:
df_train = pd.read_csv('../data_in/uplift_train.csv', index_col='client_id')
df_test = pd.read_csv('../data_in/uplift_test.csv', index_col='client_id')

In [ ]:
df_clients = pd.read_csv('../data_in/clients.csv', index_col='client_id')
df_purchases = pd.read_csv('../data_in/purchases.csv', index_col='client_id')

In [ ]:
df_products = pd.read_csv('../data_in/products.csv')

# чтение test, train с подготовлеными признаками

In [ ]:
'''
from Features_Generator import Features_Generator
#Инициализируем класс, указывая путь к папке с файлами
fg = Features_Generator('../data_in/')
df_train_f, df_test_f = fg.create_features(low_memory=True, save_files=True)
'''

In [ ]:
df_train_f = pd.read_csv('../data_in/features_set_train.csv', index_col='client_id')
df_test_f = pd.read_csv('../data_in/features_set_test.csv', index_col='client_id')

In [ ]:
df_train_f.shape, df_test_f.shape

In [69]:
# объединение данных с тренировочными и тестовыми данными
df_fix = pd.concat([df_train_f, df_test_f], ignore_index=False, sort=False)

# удаление дублирующихся строк
df_fix = del_duplicate_col(df_fix)

до: 400206
после: 400162


In [70]:
# создание категорий возрастов
df_age = f_create_agegroup(df_fix)

# очистка названий колонок от не читаемых символов
df_age = clear_bad_col(df_age)

# объединение фреймов
df_fix = pd.concat([df_fix, df_age], axis=1)

Создание категорий возрастов завершено


In [71]:
# Удаление не нужных столбцов
df_fix.drop(['treatment_flg', 'target', 'age_group_kk'], inplace=True, axis=1)

In [72]:
# нормализация количественных данных
col = df_fix.columns
df_fix = f_normaliz_df(df_fix, col[:179])

Данные нормализованы


In [ ]:
# приобретал ли алкоголь
df_fix = f_is_alhocol(df_fix)

df_fix = f_group_future(df_fix)

In [ ]:
df_ps = f_processing_purchase_sum_col(df_purchases)
df_rpr = f_processing_rpr_column(df_purchases)
df_store = f_onestore(df_purchases)

In [ ]:
###############
'''
df_ps.to_csv(f'../data_in/df_ps.csv')
df_rpr.to_csv(f'../data_in/df_rpr.csv')
df_store.to_csv(f'../data_in/df_store.csv')

df_ps = pd.read_csv('../data_in/df_ps.csv', index_col='client_id')
df_rpr = pd.read_csv('../data_in/df_rpr.csv', index_col='client_id')
df_store = pd.read_csv('../data_in/df_store.csv', index_col='client_id')
'''

# копирование подготовленного фрейма

In [ ]:
df_features = pd.concat([df_fix, df_rpr, df_store], axis=1, sort=False)
df_features.shape

# Оценка качества моделей на валидации

In [76]:
random_state = 777

In [78]:
# Модель RandomForestClassifier
model_rfc = RandomForestClassifier(random_state=random_state,
                                   max_depth=9, 
                                   min_samples_leaf=1,
                                   min_samples_split=4,
                                   n_estimators=180)

In [79]:
# Модель GradientBoostingClassifier
model_gbc = GradientBoostingClassifier(learning_rate=0.05,
                                       min_samples_leaf=14,
                                       min_samples_split=2,
                                       n_estimators=110,
                                       random_state=random_state)

'learning_rate=0.05,\n   min_samples_leaf=6,\n   min_samples_split=2,\n   n_estimators=90,\n   valid - 0.765\n   \n   learning_rate=0.05,\n   min_samples_leaf=14,\n   min_samples_split=2,\n   n_estimators=100,\n   valid - 0.793\n'

In [ ]:
# Модель XGBClassifier
model_xgbc = xgb.XGBClassifier(max_depth=10, 
                               min_child_weight=1,
                               n_estimators=400, 
                               n_jobs=-1,
                               verbose=1, 
                               learning_rate=0.15,
                               seed=42, 
                               random_state=random_state)

In [ ]:
# модель LightGBM
model_lgb = lgb.LGBMClassifier(silent=False,
                              max_depth=4,
                              learning_rate=0.01,
                              num_leaves=60,
                              n_estimators=300,
                              random_state=random_state)

# Uplift модель

In [30]:
def uplift_fit_predict(model, X_train, treatment_train, target_train, X_test):
    """
    Реализация простого способа построения uplift-модели.
    
    Обучаем два бинарных классификатора, которые оценивают вероятность target для клиента:
    1. с которым была произведена коммуникация (treatment=1)
    2. с которым не было коммуникации (treatment=0)
    
    В качестве оценки uplift для нового клиента берется разница оценок вероятностей:
    Predicted Uplift = P(target|treatment=1) - P(target|treatment=0)
    """
    X_treatment, y_treatment = X_train[treatment_train == 1, :], target_train[treatment_train == 1]
    X_control, y_control = X_train[treatment_train == 0, :], target_train[treatment_train == 0]
    model_treatment = clone(model).fit(X_treatment, y_treatment)
    model_control = clone(model).fit(X_control, y_control)
    predict_treatment = model_treatment.predict_proba(X_test)[:, 1]
    predict_control = model_control.predict_proba(X_test)[:, 1]
    predict_uplift = predict_treatment - predict_control
    
    return predict_uplift

In [31]:
def uplift_score(prediction, treatment, target, rate=0.3):
    """
    Подсчет Uplift Score
    """
    order = np.argsort(-prediction)
    treatment_n = int((treatment == 1).sum() * rate)
    treatment_p = target[order][treatment[order] == 1][:treatment_n].mean()
    control_n = int((treatment == 0).sum() * rate)
    control_p = target[order][treatment[order] == 0][:control_n].mean()
    score = treatment_p - control_p
    
    return score

# Валидация

In [80]:
indices_train = df_train.index
indices_test = df_test.index
indices_learn, indices_valid = train_test_split(df_train.index, test_size=0.33, random_state=random_state)

valid_uplift = uplift_fit_predict(model=model_gbc,
                                  X_train=df_features.loc[indices_learn, :].fillna(0).values,
                                  treatment_train=df_train.loc[indices_learn, 'treatment_flg'].values,
                                  target_train=df_train.loc[indices_learn, 'target'].values,
                                  X_test=df_features.loc[indices_valid, :].fillna(0).values,)

valid_score = uplift_score(valid_uplift,
                           treatment=df_train.loc[indices_valid, 'treatment_flg'].values,
                           target=df_train.loc[indices_valid, 'target'].values,)
print('Результат вализации:', valid_score)

Результат вализации: 0.07713872525484533


In [ ]:
'''
Validation score: GradientBoostingClassifier - 0.08173617384321996
Validation score: model_gbc - 0.0793815597055414 - 0.932
Validation score: model_gbc - 0.08130959550336248 - 0.918

Validation score: lgb.LGBMClassifier - 0.0632743393973555
Validation score: model_lgb - 0.0632743393973555

Validation score: RandomForestClassifier - 0.05228844390221066
Validation score: model_rfc - 0.059314655643286085

Validation score: xgb.XGBClassifier - 0.0820561075981131
Validation score: model_xgbc - 0.06607721690851454
'''

# Подготовка предсказаний для тестовых клиентов

In [81]:
test_uplift = uplift_fit_predict(
    model=model_gbc,
    X_train=df_features.loc[indices_train, :].fillna(0).values,
    treatment_train=df_train.loc[indices_train, 'treatment_flg'].values,
    target_train=df_train.loc[indices_train, 'target'].values,
    X_test=df_features.loc[indices_test, :].fillna(0).values,)

# Сохранение предсказания

In [82]:
date_current = datetime.today().strftime('%d_%m_%H')
df_submission = pd.DataFrame({'uplift': test_uplift}, index=df_test.index)
df_submission.to_csv(f'../data_out/submission_v5.5_{date_current}.csv')
print('file saved!')

file saved!


In [ ]:
##################
Public score = 0.930
##################